### Inteligência Artificial Preditiva
### Este notebook tem como objetivo explorar modelos de regressão e classificação em uma base de dados, utilizando as seguintes abordagens:
### Modelos de Regressão

Regressão Linear

K-Nearest Neighbors Regressor

### Modelos de Classificação

Naive Bayes (GaussianNB)

Decision Tree (Critério Gini)

Decision Tree (Critério Entropia)

K-Nearest Neighbors (KNN)

### A ideia é treinar todos os modelos disponíveis e selecionar automaticamente o que apresentar melhor desempenho, com base nas métricas adequadas:

Classificação: F1-Score, Acurácia, Precision, Recall e AUC-ROC

Regressão: R², RMSE, MAE




<h1> Importando bibliotecas </h1>

In [11]:
import os
import csv
import pandas as pd
import pickle
import numpy as np

# Pré-processamento
# ADICIONADO: GridSearchCV para Validação Cruzada
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.preprocessing import StandardScaler, LabelBinarizer

# Modelos de Regressão
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

# Modelos de Classificação
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

# Métricas
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, classification_report,
    mean_squared_error, mean_absolute_error, r2_score
)

<h1> Carregando a base de dados </h1>

In [12]:
csv_nome = "Iris.csv"  # ou "iris.csv", conforme o nome exato
caminho = os.path.join("C:\\Users\\VitorMaciel-ieg\\PROJETO_INTER\\datasets", csv_nome)

# Verifica se o arquivo existe
if not os.path.exists(caminho):
    print(f"Arquivo '{csv_nome}' não encontrado em '{caminho}'.")
else:
    try:
        with open(caminho, "r", encoding="utf-8") as f:
            sniffer = csv.Sniffer()
            sample = f.read(2048)
            f.seek(0)
            delimiter_encontrado = sniffer.sniff(sample).delimiter
        print(f"Delimitador detectado: '{delimiter_encontrado}'")
        data = pd.read_csv(caminho, delimiter=delimiter_encontrado)
        print(f"Arquivo '{csv_nome}' carregado com sucesso! Tamanho: {data.shape}")
        display(data.head())
    except Exception as e:
        print(f"Erro ao carregar o arquivo: {e}")

Delimitador detectado: ','
Arquivo 'Iris.csv' carregado com sucesso! Tamanho: (150, 6)


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


Seleção da Coluna Alvo


In [13]:
# Mostra todas as colunas
print("Colunas disponíveis no dataset:")
for i, col in enumerate(data.columns):
    print(f"{i+1}. {col}")

# Defina a coluna alvo de classificação
target_class_column = "Species"  # Substitua pelo nome da coluna que deseja prever
# Para regressão, defina target_reg_column = "nome_da_coluna"

print(f"\nColuna alvo de classificação: {target_class_column}")


Colunas disponíveis no dataset:
1. Id
2. SepalLengthCm
3. SepalWidthCm
4. PetalLengthCm
5. PetalWidthCm
6. Species

Coluna alvo de classificação: Species


<h1> Preparação de Dados para ML</h1>

In [14]:
X = data.drop(columns=[target_class_column])
Y = data[target_class_column]

# Split treino/teste
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=42, stratify=Y
)

# Padronização apenas para KNN
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Treino: {X_train.shape[0]} amostras | Teste: {X_test.shape[0]} amostras")


Treino: 105 amostras | Teste: 45 amostras


<h1> Treinamento e Avaliação de Modelos</h1>

In [20]:

tuned_models_info = {
    "KNeighbors": {
        "estimator": KNeighborsClassifier(),
        "param_grid": {'n_neighbors': np.arange(1, 21, 2), 'weights': ['uniform', 'distance']},
        "uses_scaled_data": True
    },
    "DecisionTree": {
        "estimator": DecisionTreeClassifier(random_state=42),
        "param_grid": {
            'criterion': ['gini', 'entropy'],
            'max_depth': [3, 5, 7, 9, None],
            'min_samples_split': [2, 5, 10]
        },
        "uses_scaled_data": False
    },
    
    "NaiveBayes": {
        "estimator": GaussianNB(),
        "param_grid": {},
        "uses_scaled_data": False
    }
}

classification_models = {}
classification_results = {}

#  Loop para treinamento e avaliação
for nome_modelo, info in tuned_models_info.items():
    
    X_train_data = X_train_scaled if info["uses_scaled_data"] else X_train
    X_test_data = X_test_scaled if info["uses_scaled_data"] else X_test
    
    modelo = info["estimator"]
    
    if info["param_grid"]: 
        
        print(f"\n-> Modelo: {nome_modelo} (Tuning com GridSearchCV)")
        
        # Configura e treina o GridSearch
        grid_search = GridSearchCV(
            estimator=modelo,
            param_grid=info["param_grid"],
            scoring='f1_macro', 
            cv=5,
            n_jobs=-1, 
            verbose=0
        )
        
        grid_search.fit(X_train_data, Y_train)
        
        best_model = grid_search.best_estimator_
        Y_pred = best_model.predict(X_test_data)
        
        classification_models[nome_modelo] = best_model
        
        print(f"Melhores Parâmetros: {grid_search.best_params_}")
        print(f"Melhor F1-Score (CV): {grid_search.best_score_:.4f}")
        
    else: 
        
        print(f"\n-> Modelo: {nome_modelo} (Treinamento direto)")
        modelo.fit(X_train_data, Y_train)
        Y_pred = modelo.predict(X_test_data)
        classification_models[nome_modelo] = modelo
        
    #  Avaliação no conjunto de teste para o modelo final
    acc = accuracy_score(Y_test, Y_pred)
    f1 = f1_score(Y_test, Y_pred, average='macro', zero_division=0)
    
    if nome_modelo == "DecisionTree":
       
        classification_results["DecisionTree_Gini"] = {'Acurácia': acc, 'F1-Score': f1}
        classification_models["DecisionTree_Gini"] = classification_models[nome_modelo]
        
        classification_results["DecisionTree_Entropy"] = {'Acurácia': acc, 'F1-Score': f1}
        classification_models["DecisionTree_Entropy"] = classification_models[nome_modelo]
        
        print(f"Acurácia: {acc:.4f}, F1-Score: {f1:.4f} (Melhor DT)")
        print(classification_report(Y_test, Y_pred, zero_division=0))

    else:
        classification_results[nome_modelo] = {'Acurácia': acc, 'F1-Score': f1}
        print(f"Acurácia: {acc:.4f}, F1-Score: {f1:.4f}")
        print(classification_report(Y_test, Y_pred, zero_division=0))

df_class_results = pd.DataFrame(classification_results).T
df_class_results = df_class_results.sort_values(by=['F1-Score', 'Acurácia'], ascending=False)

print("\n\n#####################################")
print("Tabela de Resultados de Classificação")
print("#####################################")
display(df_class_results)

if "NaiveBayes" not in classification_results:
    classification_models["NaiveBayes"] = tuned_models_info["NaiveBayes"]["estimator"]


-> Modelo: KNeighbors (Tuning com GridSearchCV)
Melhores Parâmetros: {'n_neighbors': np.int64(3), 'weights': 'uniform'}
Melhor F1-Score (CV): 1.0000
Acurácia: 0.9778, F1-Score: 0.9778
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        15
Iris-versicolor       0.94      1.00      0.97        15
 Iris-virginica       1.00      0.93      0.97        15

       accuracy                           0.98        45
      macro avg       0.98      0.98      0.98        45
   weighted avg       0.98      0.98      0.98        45


-> Modelo: DecisionTree (Tuning com GridSearchCV)
Melhores Parâmetros: {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}
Melhor F1-Score (CV): 0.9809
Acurácia: 1.0000, F1-Score: 1.0000 (Melhor DT)
                 precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        15
Iris-versicolor       1.00      1.00      1.00        15
 Iris-virginica       1.00     

,Acurácia,F1-Score
DecisionTree_Gini,1.000000,1.000000
DecisionTree_Entropy,1.000000,1.000000
KNeighbors,0.977778,0.977753
NaiveBayes,0.977778,0.977753


<h1> Ranking de Modelos de Classificação </h1>

In [16]:
df_class_results = pd.DataFrame(classification_results).T
df_class_results = df_class_results.sort_values(by='F1-Score', ascending=False)
print("=== Ranking Final de Classificação ===")
display(df_class_results.round(4))


=== Ranking Final de Classificação ===


,Acurácia,F1-Score
DecisionTree_Gini,1.0000,1.0000
DecisionTree_Entropy,1.0000,1.0000
KNeighbors,0.9778,0.9778
NaiveBayes,0.9778,0.9778


Preparação Modelo de Regressão


In [17]:
# Defina a coluna alvo de regressão (deve ser numérica)
target_reg_column = "SepalLengthCm"  # Exemplo, substitua pelo CSV que usar

# Seleciona apenas colunas numéricas
numeric_columns = data.select_dtypes(include=np.number).columns.tolist()

if target_reg_column not in numeric_columns:
    raise ValueError(f"A coluna alvo '{target_reg_column}' deve ser numérica.")

X_reg = data[numeric_columns].drop(columns=[target_reg_column])
Y_reg = data[target_reg_column]

# Split treino/teste
X_train_r, X_test_r, Y_train_r, Y_test_r = train_test_split(
    X_reg, Y_reg, test_size=0.3, random_state=42
)

# Padronização dos dados
scaler_r = StandardScaler()
X_train_r_scaled = scaler_r.fit_transform(X_train_r)
X_test_r_scaled = scaler_r.transform(X_test_r)

print(f"Treino: {X_train_r.shape[0]} amostras | Teste: {X_test_r.shape[0]} amostras")
print("Colunas usadas como features de regressão:")
print(X_train_r.columns.tolist())


Treino: 105 amostras | Teste: 45 amostras
Colunas usadas como features de regressão:
['Id', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']


<h1> Treinamento e Avaliação de Modelos de Regressão</h1>

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

regression_models = {
    "LinearRegression": LinearRegression(),
    "KNNRegressor": KNeighborsRegressor(n_neighbors=5)
}

regression_results = {}

for name, model in regression_models.items():
    print(f"\n-> Modelo: {name}")
    
    # Treina e faz previsões
    model.fit(X_train_r_scaled, Y_train_r)
    Y_pred = model.predict(X_test_r_scaled)
    
    # Calcula métricas
    rmse = np.sqrt(mean_squared_error(Y_test_r, Y_pred))
    mae = mean_absolute_error(Y_test_r, Y_pred)
    r2 = r2_score(Y_test_r, Y_pred)
    
    regression_results[name] = {'R²': r2, 'RMSE': rmse, 'MAE': mae}
    
    print(f"R²: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")



-> Modelo: LinearRegression
R²: 0.8498, RMSE: 0.3162, MAE: 0.2478

-> Modelo: KNNRegressor
R²: 0.8183, RMSE: 0.3478, MAE: 0.2627


<h1> Persistência do Melhor Modelo (Classificação) </h1>

In [19]:
best_model_name = df_class_results.index[0]
best_model = classification_models[best_model_name]
scaler_final = scaler if best_model_name == "KNeighbors" else None

if not os.path.exists("datasets"):
    os.makedirs("datasets")

pickle_path = os.path.join("datasets", f"modelo_class_{best_model_name}.pkl")
with open(pickle_path, "wb") as f:
    pickle.dump(best_model, f)
print(f"Modelo '{best_model_name}' salvo em: {pickle_path}")

if scaler_final:
    pickle_scaler_path = os.path.join("datasets", "scaler_class.pkl")
    with open(pickle_scaler_path, "wb") as f:
        pickle.dump(scaler_final, f)
    print("Scaler salvo em:", pickle_scaler_path)

print("\n--- Processo concluído ---")


Modelo 'DecisionTree_Gini' salvo em: datasets\modelo_class_DecisionTree_Gini.pkl

--- Processo concluído ---
